<a href="https://colab.research.google.com/github/itsual/AI-Agents/blob/main/LLAMA_Vision_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing Ollama in Colab
Follow these steps to set up your Colab environment, install Ollama using the official installation script, and verify that everything is working correctly.

## 1. Setting Up the Environment
First, install the colab-xterm package and enable the Colab XTerm extension to support command-line operations within your notebook. Then run below command

`!pip install colab-xterm`

Then

`%load_ext colabxterm`

In [19]:
# Install required packages
!pip install colab-xterm
%load_ext colabxterm

## 2. Launching XTerm
Open a full-screen terminal window within your Colab notebook by running: Run below command

`%xterm`

This will launch an interactive terminal where you can execute shell commands.

## 3. Installing Ollama
Once the xterm is open, install Ollama by executing the official installation script. Run the following command in your terminal (xterm):

`curl https://ollama.ai/install.sh | sh`

This command downloads and runs the installation script, which will automatically handle downloading and installing all necessary dependencies.

## 4. Starting the Ollama Server
After installation, start the Ollama server in the background so that it continues to run while you use your notebook:

`ollama serve &`

The & symbol runs the command in the background.

## 5. Pulling AI Models
With the server running, you can now pull AI models to use with Ollama. For example, to pull the Mistral model, run:


`ollama pull mistral`

You can replace mistral with any other model name you wish to use.

## 6. Verifying the Installation
Check that Ollama is installed correctly by verifying its version. Run:

`!ollama -version`

This command should display the version number of Ollama, confirming a successful installation.

In [22]:
%xterm

Launching Xterm...

<IPython.core.display.Javascript object>

In [23]:
!pip install ipywidgets pillow

In [24]:
import ollama
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
import io

In [30]:
# Global variable to store the current image data
current_image_content = None

# Create a FileUpload widget that accepts specific image types
upload_widget = widgets.FileUpload(
    accept='.jpg,.jpeg,.png,.webp,.tiff',
    multiple=False,
    description='Upload Image'
)

# Output widget to display the uploaded image
image_output = widgets.Output()

# Text widget for a custom prompt (default is set to ask what is in the image)
prompt_widget = widgets.Text(
    value='What is in this image?',
    description='Prompt:'
)

# Button to trigger the analysis
analyze_button = widgets.Button(
    description='Analyze Image',
    button_style='primary'
)

# Output widget to display the analysis results
analysis_output = widgets.Output()

def on_upload_change(change):
    global current_image_content
    # Clear the image display area
    image_output.clear_output()

    # Check if a file was uploaded
    if upload_widget.value:
        # There should be only one file since multiple=False
        for filename, file_info in upload_widget.value.items():
            current_image_content = file_info['content']
            try:
                # Open and display the image using PIL
                image = Image.open(io.BytesIO(current_image_content))
                with image_output:
                    display(image)
            except Exception as e:
                with image_output:
                    print("Error displaying image:", e)
        # Clear the upload widget's internal value so that a new upload replaces the old image
        upload_widget.value.clear()
        upload_widget._counter = 0

def on_analyze_clicked(b):
    # Clear any previous analysis output
    analysis_output.clear_output()
    if current_image_content is None:
        with analysis_output:
            print("Please upload an image before analyzing.")
        return

    # Save the image to disk with a fixed name (required for the Ollama call)
    file_name = "uploaded_image.jpeg"
    with open(file_name, "wb") as f:
        f.write(current_image_content)

    # Retrieve the prompt text (use custom text if provided, otherwise the default)
    prompt_text = prompt_widget.value.strip()

    with analysis_output:
        print("Sending prompt to Llama Vision via Ollama...")
        # Call the Ollama chat API with the image path provided in the 'images' list
        response = ollama.chat(
            model='llama3.2-vision',
            messages=[{
                'role': 'user',
                'content': prompt_text,
                'images': [file_name]
            }]
        )
        print("\nResponse from Llama Vision:")
        print(response)


In [31]:
# Bind the change event of the upload widget and the click event of the analyze button
upload_widget.observe(on_upload_change, names='value')
analyze_button.on_click(on_analyze_clicked)

# Display all widgets in a vertical layout
display(widgets.VBox([upload_widget, image_output, prompt_widget, analyze_button, analysis_output]))